## Setup

In [84]:
# imports
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import string
import re
import pycountry
import requests, time

# import access token
from api_key import access_token

#### Get sales df

In [4]:
# import dataset through kaggle api
api = KaggleApi()
api.authenticate()

api.dataset_download_files('kyanyoga/sample-sales-data', path='./Resources', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/kyanyoga/sample-sales-data


In [6]:
# strip trailing spaces for columns with string dtype
def strip_trailing_spaces(df):

    # make copy
    main_df = df.copy()

    # loop through columns
    for col in main_df.columns:
        # if string column then strip
        if df[col].dtype == 'object':
            df[col] = main_df[col].str.strip()

    # return df
    return main_df

In [7]:
# convert to df
sales_df = pd.read_csv('Resources/sales_data_sample.csv', encoding='latin1')

# strip leading/trailing spaces
sales_df = strip_trailing_spaces(sales_df)

# display df
print(sales_df.shape)
sales_df.head()

(2823, 25)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Data Check
---

### Quick checks

In [8]:
# check nulls
sales_df.isna().sum()

ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE1           0
ADDRESSLINE2        2521
CITY                   0
STATE               1486
POSTALCODE            76
COUNTRY                0
TERRITORY           1074
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64

In [9]:
# look at unique count
sales_df.nunique()

ORDERNUMBER          307
QUANTITYORDERED       58
PRICEEACH           1016
ORDERLINENUMBER       18
SALES               2763
ORDERDATE            252
STATUS                 6
QTR_ID                 4
MONTH_ID              12
YEAR_ID                3
PRODUCTLINE            7
MSRP                  80
PRODUCTCODE          109
CUSTOMERNAME          92
PHONE                 91
ADDRESSLINE1          92
ADDRESSLINE2           9
CITY                  73
STATE                 16
POSTALCODE            73
COUNTRY               19
TERRITORY              3
CONTACTLASTNAME       77
CONTACTFIRSTNAME      72
DEALSIZE               3
dtype: int64

In [10]:
# data types check
sales_df.dtypes

ORDERNUMBER           int64
QUANTITYORDERED       int64
PRICEEACH           float64
ORDERLINENUMBER       int64
SALES               float64
ORDERDATE            object
STATUS               object
QTR_ID                int64
MONTH_ID              int64
YEAR_ID               int64
PRODUCTLINE          object
MSRP                  int64
PRODUCTCODE          object
CUSTOMERNAME         object
PHONE                object
ADDRESSLINE1         object
ADDRESSLINE2         object
CITY                 object
STATE                object
POSTALCODE           object
COUNTRY              object
TERRITORY            object
CONTACTLASTNAME      object
CONTACTFIRSTNAME     object
DEALSIZE             object
dtype: object

### Create datetime column

In [11]:
# convert orderdate
sales_df['ORDERDATE'] = pd.to_datetime(sales_df['ORDERDATE'])

# check dtypes
sales_df.dtypes

ORDERNUMBER                  int64
QUANTITYORDERED              int64
PRICEEACH                  float64
ORDERLINENUMBER              int64
SALES                      float64
ORDERDATE           datetime64[ns]
STATUS                      object
QTR_ID                       int64
MONTH_ID                     int64
YEAR_ID                      int64
PRODUCTLINE                 object
MSRP                         int64
PRODUCTCODE                 object
CUSTOMERNAME                object
PHONE                       object
ADDRESSLINE1                object
ADDRESSLINE2                object
CITY                        object
STATE                       object
POSTALCODE                  object
COUNTRY                     object
TERRITORY                   object
CONTACTLASTNAME             object
CONTACTFIRSTNAME            object
DEALSIZE                    object
dtype: object

In [12]:
# order data by datetime
sales_df = sales_df.sort_values(by='ORDERDATE')

## Fix Addresses
---

### Check for duplicate naming issues (ex: "Company Inc." and "company inc")

In [15]:
# list of cols that could be duplicates (object type)
dupe_cols_check = list(sales_df.select_dtypes(include=['object']).columns)

# check for dupes
for col in dupe_cols_check:
    # create dupe col check
    col_check = f'{col}_CHECK'
    sales_df[col_check] = sales_df[col].str.title().str.translate(str.maketrans('', '', string.punctuation))

    # check if naming is already unique, if not use new col
    if sales_df[col_check].nunique() == sales_df[col].nunique():
        # drop new col
        sales_df = sales_df.drop(columns=[col_check]).copy()
        print(f'{col}: no duplicate naming issues')
    else:
        # drop old col
        sales_df = sales_df.drop(columns=[col]).copy()
        print(f'{col}: replaced column - found duplicate naming issues!!!')

STATUS: no duplicate naming issues
PRODUCTLINE: no duplicate naming issues
PRODUCTCODE: no duplicate naming issues
CUSTOMERNAME: no duplicate naming issues
PHONE: no duplicate naming issues
ADDRESSLINE1: no duplicate naming issues
ADDRESSLINE2: no duplicate naming issues
CITY: no duplicate naming issues
STATE: no duplicate naming issues
POSTALCODE: no duplicate naming issues
COUNTRY: no duplicate naming issues
TERRITORY: no duplicate naming issues
CONTACTLASTNAME: no duplicate naming issues
CONTACTFIRSTNAME: no duplicate naming issues
DEALSIZE: no duplicate naming issues


### Check if full addresses are unique

In [16]:
# check if only one address option for each unique addressline
nunique_address = sales_df.groupby([
    'ADDRESSLINE1',
    'CITY',
    'COUNTRY'
]).nunique()

# loop through to see if any address has multiple of any of these fields
for col in ['ADDRESSLINE2', 'STATE', 'POSTALCODE', 'TERRITORY']:
    multiple_cnt = len(nunique_address[nunique_address[col] > 1])
    if multiple_cnt > 0:
        print(f'{col}: NOT unique -- multiples count of {multiple_cnt}')
    else:
        print(f'{col}: unique')

ADDRESSLINE2: unique
STATE: unique
POSTALCODE: unique
TERRITORY: unique


### Create customer ID

In [17]:
# groupby customer name
sales_df['CUSTOMER_ID'] = sales_df.groupby([
    'CUSTOMERNAME',
    'CONTACTLASTNAME', 
    'CONTACTFIRSTNAME',
    # 'ADDRESSLINE1',
    # 'ADDRESSLINE2',
    # 'CITY',
    # 'STATE',
    # 'POSTALCODE',
    # 'COUNTRY',
    # 'TERRITORY'
]).ngroup() + 1

sales_df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,CUSTOMER_ID
578,10100,30,100.00,3,5151.00,2003-01-06,Shipped,1,1,2003,...,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Medium,61
680,10100,50,67.80,2,3390.00,2003-01-06,Shipped,1,1,2003,...,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Medium,61
1267,10100,22,86.51,4,1903.22,2003-01-06,Shipped,1,1,2003,...,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Small,61
2024,10100,49,34.47,1,1689.03,2003-01-06,Shipped,1,1,2003,...,NaN,Nashua,NH,62005,USA,NaN,Young,Valarie,Small,61
1539,10101,45,31.20,3,1404.00,2003-01-09,Shipped,1,1,2003,...,NaN,Frankfurt,NaN,60528,Germany,EMEA,Keitel,Roland,Small,14


In [18]:
# check count of unique customers
print(sales_df.CUSTOMERNAME.nunique())
print(sales_df.CUSTOMER_ID.nunique())

92
92


### Get addresses (one address associated with one customer)

In [19]:
# function to pick a non null value from each col --> just in case
def pick_first_notna(col):
    # drop nulls
    not_na = col.dropna()

    # return first non null value if present
    return not_na.iloc[0] if len(not_na) > 0 else None

In [20]:
# group addresses by customer id and get values for as many columns
address_df = sales_df.groupby('CUSTOMER_ID', as_index=False).agg({
    'ADDRESSLINE1': pick_first_notna,
    'ADDRESSLINE2': pick_first_notna,
    'CITY': pick_first_notna,
    'STATE': pick_first_notna,
    'POSTALCODE': pick_first_notna,
    'COUNTRY': pick_first_notna,
    'TERRITORY': pick_first_notna
})

In [21]:
# display length and nulls
print(f'Number of rows: {len(address_df)}')
address_df.isna().sum()

Number of rows: 92


CUSTOMER_ID      0
ADDRESSLINE1     0
ADDRESSLINE2    83
CITY             0
STATE           46
POSTALCODE       3
COUNTRY          0
TERRITORY       38
dtype: int64

### Drop territory column

In [22]:
# look into territories
print(f'Nulls: {address_df.TERRITORY.isna().sum()}')
address_df.TERRITORY.value_counts()

Nulls: 38


TERRITORY
EMEA     44
APAC      6
Japan     4
Name: count, dtype: int64

In [23]:
# drop
address_df = address_df.drop(columns=['TERRITORY']).copy()
address_df.head(3)

,CUSTOMER_ID,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY
0,1,Fauntleroy Circus,None,Manchester,None,EC2 5NT,UK
1,2,1 rue Alsace-Lorraine,None,Toulouse,None,31000,France
2,3,Via Monte Bianco 34,None,Torino,None,10100,Italy


### Build single address column

In [24]:
# # function to build a single address string/col
# def build_full_address(row):
#     parts = []
    
#     for col in ['ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE', 'COUNTRY']:
#         if pd.notnull(row[col]):
#             parts.append(row[col])
#     return ', '.join(parts)

In [25]:
# # create single address column
# address_df['FULL_ADDRESS'] = address_df.apply(build_full_address, axis=1)
# address_df.head()

### Get country codes

In [52]:
# update abbreviated countries
address_df['COUNTRY'] = address_df['COUNTRY'].replace({
    'UK': 'United Kingdom',
    'USA': 'United States'
})
address_df.COUNTRY.unique()

array(['United Kingdom', 'France', 'Italy', 'Australia', 'United States',
       'Norway', 'Germany', 'Spain', 'Canada', 'Ireland', 'Philippines',
       'Denmark', 'Singapore', 'Austria', 'Japan', 'Finland', 'Belgium',
       'Sweden', 'Switzerland'], dtype=object)

In [59]:
# get three character country code
countries = address_df.COUNTRY.unique()

country_code_dict = {
    country: pycountry.countries.get(name=country).alpha_3
    for country in countries
}

address_df['COUNTRY_CODE'] = address_df['COUNTRY'].map(country_code_dict)

# display
address_df.head(3)

,CUSTOMER_ID,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,COUNTRY_CODE
0,1,Fauntleroy Circus,None,Manchester,None,EC2 5NT,United Kingdom,GBR
1,2,1 rue Alsace-Lorraine,None,Toulouse,None,31000,France,FRA
2,3,Via Monte Bianco 34,None,Torino,None,10100,Italy,ITA


In [ ]:
# double check codes
country_code_dict

{'United Kingdom': 'GBR',
 'France': 'FRA',
 'Italy': 'ITA',
 'Australia': 'AUS',
 'United States': 'USA',
 'Norway': 'NOR',
 'Germany': 'DEU',
 'Spain': 'ESP',
 'Canada': 'CAN',
 'Ireland': 'IRL',
 'Philippines': 'PHL',
 'Denmark': 'DNK',
 'Singapore': 'SGP',
 'Austria': 'AUT',
 'Japan': 'JPN',
 'Finland': 'FIN',
 'Belgium': 'BEL',
 'Sweden': 'SWE',
 'Switzerland': 'CHE'}

### Validate addresses with ArcGIS geocoding service api

In [61]:
address_df.isna().sum()

CUSTOMER_ID      0
ADDRESSLINE1     0
ADDRESSLINE2    83
CITY             0
STATE           46
POSTALCODE       3
COUNTRY          0
COUNTRY_CODE     0
dtype: int64

In [ ]:
# ArcGIS REST API geocoding service url
api_url = 'https://geocode-api.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates'

In [ ]:
# test df
two_addresses = address_df.head(2).copy()
two_addresses

,CUSTOMER_ID,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,COUNTRY_CODE
0,1,Fauntleroy Circus,None,Manchester,None,EC2 5NT,United Kingdom,GBR
1,2,1 rue Alsace-Lorraine,None,Toulouse,None,31000,France,FRA


In [93]:
# output fields
out_fields = ','.join([
    'StAddr',
    'SubAddr',
    'City',
    'Region',
    'RegionAbbr',
    'Subregion',
    'Postal',
    #'Country',
    'CntryName',
    'CountryCode',
    'Addr_type', 'Match_addr', 'Status'  #quality/diagnostics
])

In [ ]:
# function to return '' for None address fields
def none_to_empty(field):
    return '' if field is None else field

# function to get params
def build_params(row):

    # setup mulitfield query (return nothing for null fields)
    query = {
        'address': none_to_empty(row.ADDRESSLINE1),
        'address2': none_to_empty(row.ADDRESSLINE2),
        'city': none_to_empty(row.CITY),
        'region': none_to_empty(row.STATE),
        'postal': none_to_empty(row.POSTALCODE),
        'countryCode': none_to_empty(row.COUNTRY_CODE)
    }

    # drop blanks from query 
    query = {field: input for field, input in query.items() if input}

    base = {
        'f': 'pjson',
        'token': access_token,
        'forStorage': 'false',
        'langCode': 'ENG',
        'outFields': out_fields
    }

    base.update(query)

    # return parameters
    return base 

In [97]:
# get official/valid addresses
val_addresses = []

for _, row in address_df.iterrows():
    
    # get params
    params_dict = build_params(row)
    
    try:
        # make api request
        address_data = requests.get(api_url, params=params_dict, timeout=10).json()
        print(address_data)
        
        candidate = address_data.get('candidates', [None])[0]
        time.sleep(15)

        # build dict
        if candidate:
            attr = candidate['attributes']
            val_addresses.append({
                'CUSTOMER_ID'     : row.CUSTOMER_ID,
                'STD_ADDRESSLINE1': attr['StAddr']      or pd.NA,
                'STD_ADDRESSLINE2': attr['SubAddr']     or pd.NA,
                'STD_CITY'        : attr['City']        or pd.NA,
                'STD_REGION'      : attr['Region']      or pd.NA,
                'STD_REGIONCODE'  : attr['RegionAbbr']  or pd.NA,
                'STD_POSTALCODE'  : attr['Postal']      or pd.NA,
                'STD_COUNTRY'     : attr['CntryName']   or pd.NA,
                'STD_COUNTRY_CODE': attr['CountryCode'] or pd.NA,
                'ADDR_TYPE'       : attr['Addr_type']   or pd.NA,
                'MATCH_ADDR'      : attr['Match_addr']  or pd.NA,
                'STATUS'          : attr['Status']      or pd.NA,
                'SCORE'           : candidate['score']  or pd.NA
            })
    
    # display errors
    except requests.RequestException as e:
        print(f'Error for {row.CUSTOMER_ID}: {e}')
        print(f'Error type: {type(e)}')
        print(f'Arguments: {e.args}')
    except Exception as e:
        print(f'Unexpected error for {row.CUSTOMER_ID}: {e}')
        print(f'Error type: {type(e)}')
        print(f'Arguments: {e.args}')
    

{'spatialReference': {'wkid': 4326, 'latestWkid': 4326}, 'candidates': [{'address': 'Manchester, Greater Manchester, England', 'location': {'x': -2.245275942, 'y': 53.47895622}, 'score': 71.5, 'attributes': {'Status': 'T', 'Match_addr': 'Manchester, Greater Manchester, England', 'Addr_type': 'Locality', 'SubAddr': '', 'StAddr': '', 'City': 'Manchester', 'Subregion': 'Greater Manchester', 'Region': 'England', 'RegionAbbr': '', 'Postal': '', 'CntryName': 'United Kingdom', 'CountryCode': 'GBR'}, 'extent': {'xmin': -2.318275942, 'ymin': 53.40595622, 'xmax': -2.172275942, 'ymax': 53.55195622}}, {'address': 'Manchester, Lancashire, England', 'location': {'x': -2.2343765, 'y': 53.4807125}, 'score': 71.5, 'attributes': {'Status': 'T', 'Match_addr': 'Manchester, Lancashire, England', 'Addr_type': 'Locality', 'SubAddr': '', 'StAddr': '', 'City': 'Manchester', 'Subregion': 'Lancashire', 'Region': 'England', 'RegionAbbr': 'ENG', 'Postal': '', 'CntryName': 'United Kingdom', 'CountryCode': 'GBR'}, '

In [98]:
# create dataframe
val_address_df = pd.DataFrame(val_addresses)
val_address_df

,CUSTOMER_ID,STD_ADDRESSLINE1,STD_ADDRESSLINE2,STD_CITY,STD_REGION,STD_REGIONCODE,STD_POSTALCODE,STD_COUNTRY,STD_COUNTRY_CODE,ADDR_TYPE,MATCH_ADDR,STATUS,SCORE
0,1,<NA>,<NA>,Manchester,England,<NA>,<NA>,United Kingdom,GBR,Locality,"Manchester, Greater Manchester, England",T,71.50
1,2,1 Rue d'Alsace Lorraine,<NA>,Toulouse,Occitanie,<NA>,31000,France,FRA,PointAddress,"1 Rue d'Alsace Lorraine, 31000, Toulouse, Haut...",M,100.00
2,3,Via Carlo Bianco 34,<NA>,Torino,Piemonte,<NA>,10146,Italia,ITA,StreetAddressExt,"Via Carlo Bianco 34, 10146, Torino",M,93.22
3,4,201 Miller St,L 15,Sydney,New South Wales,NSW,2060,Australia,AUS,Subaddress,"201 Miller St, L 15, North Sydney, Sydney, New...",M,100.00
4,5,54 Place Royale,<NA>,Nantes,Pays de la Loire,PDL,44000,France,FRA,StreetAddressExt,"54 Place Royale, 44000, Nantes, Loire-Atlantiq...",M,96.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,87,12 Barkeley Drive,<NA>,Liverpool,England,ENG,L21 4LX,United Kingdom,GBR,PointAddress,"12 Barkeley Drive, Seaforth, Liverpool, Mersey...",M,80.14
87,88,<NA>,<NA>,Geneva,Geneva,GE,1203,Switzerland,CHE,Postal,"1203, Geneva",M,71.87
88,89,<NA>,<NA>,New York,New York,NY,10022,United States,USA,Postal,"10022, New York, New York",M,84.00
89,91,<NA>,<NA>,Half Moon Bay,California,CA,94019,United States,USA,Postal,"94019, Half Moon Bay, California",M,77.00


In [ ]:
# save output
val_address_df.to_csv('Output/val_address.csv', index=False, encoding='latin1')

In [ ]:
# check address with error
address_df[address_df.CUSTOMER_ID == 90]

,CUSTOMER_ID,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,COUNTRY_CODE
89,90,Berguvsvgen 8,None,Lule,None,S-958 22,Sweden,SWE


## Create Relational Tables
---

### Customer Table